## Full Keras with augmentation

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import skimage 
from scipy import ndimage, misc

In [3]:
import cv2
def plot_image(file_path):
    """ Print image from file 'file_path' """
    img = cv2.imread(file_path)
    cv_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(cv_rgb)
    plt.show()

In [4]:
from keras.utils import np_utils
from sklearn.datasets import load_files

data = load_files('prep/train')
caltech_files_train = np.array(data['filenames'])
caltech_targets_train = np.array(data['target'])

data_val = load_files('prep/val')
caltech_files_val = np.array(data_val['filenames'])
caltech_targets_val = np.array(data_val['target'])

X_train, X_val = caltech_files_train, caltech_files_val
y_train_class, y_val_class = caltech_targets_train, caltech_targets_val
y_train = np_utils.to_categorical(y_train_class, 257)
y_val = np_utils.to_categorical(y_val_class, 257)

del data, data_val, caltech_files_train, caltech_targets_train, caltech_files_val, caltech_targets_val

Using TensorFlow backend.


In [5]:
test_files = ['prep/test/'+ file for file in os.listdir('prep/test')]
#test_files[:5]

In [6]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

resize = (64, 64)

CHANNELS = 3
ROWS = resize[0]
COLS = resize[1]
mean = 135

datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

## Data load and resize

In [7]:
from keras.preprocessing import image                  
from tqdm import tqdm

def path_to_tensor(img_path, resize = resize):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=resize)
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths, resize = resize):
    list_of_tensors = [path_to_tensor(img_path, resize) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In [17]:
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True                 

# pre-process the data for Keras
train_data = paths_to_tensor(X_train, resize).astype('float32')/255
valid_data = paths_to_tensor(X_val, resize).astype('float32')/255
test_data = paths_to_tensor(test_files, resize).astype('float32')/255

100%|██████████| 7710/7710 [00:02<00:00, 2580.14it/s]


In [18]:
print(train_data.shape)
print(valid_data.shape)
print(test_data.shape)

(16027, 64, 64, 3)
(6870, 64, 64, 3)
(7710, 64, 64, 3)


In [10]:
batch_size = 16

train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        shear_range=0.2,
        zoom_range=0.2,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True)

# only rescaling for test set
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        'prep/train',  # this is the target directory
        target_size=(ROWS, COLS),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='categorical')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = train_datagen.flow_from_directory(
        'prep/val',
        target_size=(ROWS, COLS),
        batch_size=batch_size,
        class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    'prep/test',
    target_size=(64, 64),
    class_mode=None,
    batch_size=64,
    shuffle=False)

Found 16027 images belonging to 257 classes.
Found 6870 images belonging to 257 classes.
Found 0 images belonging to 0 classes.


In [11]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling1D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential

model = Sequential()

model.add(Conv2D(filters=16, kernel_size=2, padding='same', activation='relu', 
                        input_shape=(resize[0], resize[1], CHANNELS)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
#model.add(Conv2D(filters=128, kernel_size=3, padding='same', activation='relu'))
#model.add(MaxPooling2D(pool_size=2))
#model.add(Conv2D(filters=256, kernel_size=3, padding='same', activation='relu'))
#model.add(MaxPooling2D(pool_size=2))
#model.add(Conv2D(filters=512, kernel_size=3, padding='same', activation='relu'))
#model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(257, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 64, 16)        208       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        2080      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 64)        8256      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 8, 8, 64)          0         
__________

In [12]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

epochs = 20

checkpointer = [ModelCheckpoint(filepath='saved_models/weights.best.keras_aug1.hdf5', 
                               verbose=1, save_best_only=True),
               ReduceLROnPlateau(verbose=1, patience=3, factor = 0.1), 
               EarlyStopping(verbose=1, patience=5, monitor='val_loss')]

#model.fit(train, y_train, 
#          validation_data=(val, y_val),
#          epochs=epochs, batch_size=64, callbacks=checkpointer, verbose=1)

In [14]:
model.fit_generator(
        train_generator,
        steps_per_epoch=len(train_data) / batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=len(valid_data) / batch_size, callbacks=checkpointer, verbose=1)

Epoch 1/20
1002/1001 [==============================] - 28s - loss: 5.2263 - acc: 0.0683 - val_loss: 4.9153 - val_acc: 0.0964
Epoch 2/20
1002/1001 [==============================] - 25s - loss: 4.8064 - acc: 0.1092 - val_loss: 4.5778 - val_acc: 0.1320
Epoch 3/20
1002/1001 [==============================] - 26s - loss: 4.5636 - acc: 0.1320 - val_loss: 4.4321 - val_acc: 0.1572
Epoch 4/20
1002/1001 [==============================] - 26s - loss: 4.4112 - acc: 0.1533 - val_loss: 4.2815 - val_acc: 0.1713
Epoch 5/20
1002/1001 [==============================] - 26s - loss: 4.2958 - acc: 0.1646 - val_loss: 4.2022 - val_acc: 0.1859
Epoch 6/20
1002/1001 [==============================] - 26s - loss: 4.1983 - acc: 0.1716 - val_loss: 4.1493 - val_acc: 0.1911
Epoch 7/20
1002/1001 [==============================] - 26s - loss: 4.1073 - acc: 0.1862 - val_loss: 4.0447 - val_acc: 0.2066
Epoch 8/20
1002/1001 [==============================] - 26s - loss: 4.0327 - acc: 0.1847 - val_loss: 4.0124 - val_acc:

In [19]:
model.load_weights('saved_models/weights.best.keras_aug1.hdf5')
predictions = [np.argmax(model.predict(np.expand_dims(feature, axis=0))) for feature in valid_data]
# get val accuracy
val_accuracy = 100*np.sum(np.array(predictions)==np.argmax(y_val, axis=1))/len(predictions)
print('Validation accuracy: %.2f%%' % val_accuracy)

Validation accuracy: 28.17%


In [21]:
predictions_test = [1+np.argmax(model.predict(np.expand_dims(feature, axis=0))) for feature in test_data]

In [22]:
image = [item.split('/')[1] for item in test_files]
submission = pd.DataFrame(
    {'image': image,
     'class': predictions_test
    })
submission = submission[['image','class']]
submission.head()

,image,class
0,test,105
1,test,232
2,test,149
3,test,232
4,test,38


In [23]:
submission.to_csv('keras_aug.csv', index = False)